<a href="https://colab.research.google.com/github/woels/openmv/blob/master/docs/files/LeRobot_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# LeRobot Training Instruction Manual

# 🎉 Welcome to the LeRobot Training Notebook!

This guide will help you set up and train a model on a cloud-based platform, such as **Google Colab**, using **LeRobot** with **Hugging Face**.

---

## ⚠️ **Disclaimers:**

- **GPU Subscription**: 🔑 Make sure you have the appropriate subscription plan that provides access to the necessary GPU (e.g., **A100**, **T4**). Review pricing and benefits on the cloud provider's website before proceeding.
  
- **Checkpoint Requirement**: ⏳ If resuming training, ensure that you have the previous training checkpoint available in your session. Without the checkpoint, the training **cannot be resumed**.

---

## 📝 **Important Instructions:**

- **Run All Cells Together**: 🔄 It is recommended to run all the cells in one go if you plan to leave the session **unmonitored**. This helps avoid session timeouts or disruptions.

- **GPU & Compute Units**: 🎛️ Ensure you select a suitable GPU (e.g., **A100**, **T4**) and have enough compute units for your session. A typical 5-hour training session requires approximately **70 compute units**.

- **Monitor Training**: 👀 It’s advisable to monitor the **first few epochs** to ensure that the training is running smoothly before leaving the session unattended.

- **Local Storage**: 💾 You will be prompted to choose whether you want to store the training outputs **locally** at the end of the process.

---

Now, let’s begin the setup process! 🚀


---


In [ ]:
# Collect all necessary inputs from the user
import os
import subprocess

# GPU selection (Reminder: Ensure enough compute units for smooth training)
print("Please select a suitable GPU type (e.g., A100, T4) for cloud-based training.")

# Hugging Face login token
print("Generate a Hugging Face token from: https://huggingface.co/settings/tokens")
hf_token = input("Please enter your Hugging Face token: ")

# Link to Trossen Robotics Community datasets
print("You can explore datasets from Trossen Robotics Community here: https://huggingface.co/TrossenRoboticsCommunity")

# Dataset and job details
dataset_repo_id = input("Please enter the dataset repo ID from Hugging Face (e.g., TrossenRoboticsCommunity/aloha_stationary_logo_assembly): ")


print("**Important**: Use a valid directory/jobs name. Avoid numbers or special characters other than '_'.")
print("Example: 'training_results_aloha' or 'aloha_training_output'")

job_name = input("Please enter the job name for this training session: ")

# Output directory with naming format instructions

output_dir = input("Enter the output directory name: ")

# Resume flag with disclaimer
resume_flag = input("Do you want to resume training from a previous checkpoint? (yes/no): ")
resume_cmd = "--resume" if resume_flag.lower() == 'yes' else ""

# Model upload flag
upload_choice = input("Do you want to upload the model to Hugging Face after training? (yes/no): ")
model_repo_id = ""
if upload_choice.lower() == 'yes':
    model_repo_id = input("Please enter the model repo ID to store your trained model to Hugging Face (e.g., TrossenRoboticsCommunity/aloha_stationary_logo_assembly): ")

# Local storage flag
store_locally = input("Do you want to store the training outputs locally? (yes/no): ")



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## Step 1: GPU Setup & Compute Units

The GPU type you selected earlier will now be configured for this cloud-based training session. Make sure to have enough compute units to support long training sessions, and monitor the first few epochs to ensure smooth execution.

---



## Step 2: Installing Dependencies

In this step, we'll install all the necessary dependencies for running LeRobot and performing model training.

Ensure that these packages are successfully installed before proceeding to the next steps.

---


In [ ]:

# Install required dependencies
print("Installing required dependencies...")

!sudo apt-get install libusb-1.0-0-dev
!pip install --upgrade  pyrealsense2 dynamixel-sdk rerun-sdk blinker wandb datasets huggingface-hub hydra-core gitpython flask diffusers InquirerPy

# Install blinker if needed
!pip install --ignore-installed blinker


# Install LeRobot repository
!git clone https://github.com/huggingface/lerobot.git
%cd /content/lerobot
!ls
!pip install -e .
!pip install .[intelrealsense,dynamixel]

print("Dependencies installed successfully.")



## Step 3: Hugging Face Login & Dataset Setup

We will now log into Hugging Face using the token provided. After login, the dataset repo, job name, and output directory that you specified will be configured for the training session.

---


In [ ]:
# Log in to Hugging Face and verify login
print("Logging into Hugging Face...")
!huggingface-cli login --token {hf_token}

# Verify the login by checking the user information
user_info = !huggingface-cli whoami
print(f"Logged in as: {user_info[0]}")


> **⚠️ Important Notice:**
>
> Before you start the training, make sure to edit the `act_aloha_real.yaml` file located at:
>
> **Click Here** >> `/content/lerobot/lerobot/configs/policy/act_aloha_real.yaml`
>
> This file contains crucial parameters such as `batch_size`, `offline_steps`, and `learning_rate`. You should update these parameters based on your training needs. For example, you can modify:
>
> - **Batch Size** (`training.batch_size`): Adjust the number of samples processed in each training step.
> - **Offline Training Steps** (`training.offline_steps`): Define how many steps to run during offline training.
> - **Learning Rate** (`training.lr`): Set the learning rate to control how quickly the model learns.
>
> Once the file is updated, you can proceed with training.


## Step 5: Model Training or Resumption

Now that everything is set up, we can either begin training the model or resume training from the last checkpoint, depending on your input.

If resuming, make sure the checkpoint is available in your session. The training will continue from the last checkpoint if found.

> **⚠️ Important: GPU Usage**
>
> By default, the training is configured to use a **GPU** for faster computation. If the runtime does not have access to a GPU, the training will fail.
>
> To avoid this issue:
>
> - **Ensure GPU is enabled** in your Colab runtime. You can check this by navigating to **Runtime > Change runtime type > Hardware accelerator** and selecting **GPU**.
> - If you prefer to use a **CPU** instead, update the `device` argument to `device=cpu` in the training command in the next cell.

---

In [ ]:
# Start or resume training depending on user choice
%cd /content/lerobot/

if resume_flag.lower() == "no":
    print(f"Starting new training on {dataset_repo_id}...")
    !python lerobot/scripts/train.py dataset_repo_id={dataset_repo_id} policy=act_aloha_real env=aloha_real hydra.run.dir=outputs/train/{output_dir} hydra.job.name={job_name} device=cuda wandb.enable=false
else:
    print(f"Resuming training from {output_dir}... (ensure checkpoint is available)")
    !python lerobot/scripts/train.py hydra.run.dir={output_dir} resume=true



## Step 5: Uploading the Model (Recommended)

Once the model is trained, you can choose to upload it to Hugging Face for safekeeping. This is **highly recommended** if you are running long sessions or training a valuable model.

Uploading the model will help protect against potential session interruptions or failures.

---


In [ ]:
print(model_repo_id)
# Model upload step if chosen
if upload_choice.lower() == "yes":
    print("Uploading the model to Hugging Face...")
    !huggingface-cli upload {model_repo_id}  outputs/train/{output_dir}/checkpoints/last/pretrained_model
    print("Model uploaded to Hugging Face successfully.")
else:
    print("Model upload skipped.")




## Step 6: Safeguarding Session Data and Local Storage

To prevent data loss in case of session termination, you can zip the output directory and download it locally. If you selected local storage, the outputs will be saved to your local machine.

Make sure to run this step to save all training outputs before closing your session.

---


In [ ]:
# Zip the output directory and download it if local storage is chosen
%cd /content/lerobot/outputs/train/
!ls
if store_locally.lower() == "yes":
    print("Zipping outputs for download...")
    !zip -r trained.zip {output_dir}

    # Download the zipped file
    from google.colab import files
    files.download('/content/lerobot/outputs/train/trained.zip')
else:
    print("Local storage not selected, skipping download.")


# Troubleshooting and Recommendations

1. **GPU Availability**: Ensure the selected GPU is available on your cloud platform (e.g., Colab).
2. **Compute Units**: Ensure you have sufficient compute units. Each 5-hour session requires ~70 units.
3. **Hugging Face Token**: You can generate a token [here](https://huggingface.co/settings/tokens).
4. **Session Safeguards**: Always download your results (output files) to prevent data loss if the session terminates.
5. **Checkpoint Reminder**: If resuming training, ensure that the checkpoint file from the previous session is present in the session.

---
